In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,7)
from datetime import datetime, timedelta
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
import math
import textblob as TextBlob
from textblob import TextBlob
import preprocessor as p

In [3]:
tweets1 = pd.read_csv('tweetsfinal1.csv', encoding='ISO-8859-1', error_bad_lines = False, engine = 'python', header = None)
tweets2 = pd.read_csv('tweetsfinal2.csv', encoding='ISO-8859-1', error_bad_lines=False,engine = 'python',header = None)
tweets3 = pd.read_csv('tweetsfinal3.csv', encoding = 'ISO-8859-1', error_bad_lines = False, engine = 'python', header = None)

Skipping line 845142: unexpected end of data
Skipping line 352574: NULL byte detected. This byte cannot be processed in Python's native csv library at the moment, so please pass in engine='c' instead
Skipping line 353631: NULL byte detected. This byte cannot be processed in Python's native csv library at the moment, so please pass in engine='c' instead
Skipping line 883146: unexpected end of data


In [4]:
tweets2 = tweets2.drop([2,3], axis=1)
tweets3 = tweets3.drop([2,3], axis=1)

In [48]:
tweetsDF = pd.concat([tweets1,tweets2,tweets3], ignore_index = True)

In [49]:
tweetsDF

,0,1
0,Thu Nov 09 17:43:41 EST 2017,RT @Forbes: The Failure of SegWit2x Shows Bitc...
1,Thu Nov 09 17:43:40 EST 2017,RT @mindstatex: Lots of love from unknown mine...
2,Thu Nov 09 17:43:39 EST 2017,RT @FernandoHuamanX: Warning: Built-in Keylogg...
3,Thu Nov 09 17:43:39 EST 2017,RT @LevelNetwork: Join our telegram. All infor...
4,Thu Nov 09 17:43:39 EST 2017,"RT @realsheepwolf: $DIGAF: FLOAT=16M, THE ""ONL..."
...,...,...
2564345,Mon Nov 20 13:40:17 EST 2017,"Bitcoin continues to crush it, breaks through ..."
2564346,Mon Nov 20 13:40:16 EST 2017,RT @charliebilello: Bitcoin's market cap just ...
2564347,Mon Nov 20 13:40:16 EST 2017,#CheckItOut as #AML bitcoin looks to test the ...
2564348,Mon Nov 20 13:40:16 EST 2017,CME's Bitcoin Futures Likely to Start Trading ...


In [50]:
tweetsDF.dropna()

,0,1
0,Thu Nov 09 17:43:41 EST 2017,RT @Forbes: The Failure of SegWit2x Shows Bitc...
1,Thu Nov 09 17:43:40 EST 2017,RT @mindstatex: Lots of love from unknown mine...
2,Thu Nov 09 17:43:39 EST 2017,RT @FernandoHuamanX: Warning: Built-in Keylogg...
3,Thu Nov 09 17:43:39 EST 2017,RT @LevelNetwork: Join our telegram. All infor...
4,Thu Nov 09 17:43:39 EST 2017,"RT @realsheepwolf: $DIGAF: FLOAT=16M, THE ""ONL..."
...,...,...
2564345,Mon Nov 20 13:40:17 EST 2017,"Bitcoin continues to crush it, breaks through ..."
2564346,Mon Nov 20 13:40:16 EST 2017,RT @charliebilello: Bitcoin's market cap just ...
2564347,Mon Nov 20 13:40:16 EST 2017,#CheckItOut as #AML bitcoin looks to test the ...
2564348,Mon Nov 20 13:40:16 EST 2017,CME's Bitcoin Futures Likely to Start Trading ...


In [51]:
tweetsDF.columns = ['DateTime','Tweet']

In [52]:
# import re
# def cleanTxt(text):
#     text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    
#     return text

# tweetsDF['Tweet'] = tweetsDF['Tweet'].apply(cleanTxt)

# tweetsDF

In [53]:
# tweetsDF["Tweet"] = tweetsDF["Tweet"].str.replace("RT","")

In [54]:
import preprocessor as p
def preprocess_tweet(row):
    Tweet = row['Tweet']
    Tweet = p.clean(Tweet)
    return Tweet
tweetsDF['Tweet'] = tweetsDF.apply(preprocess_tweet, axis=1)

In [55]:
tweetsDF

,DateTime,Tweet
0,Thu Nov 09 17:43:41 EST 2017,: The Failure of SegWit2x Shows Bitcoin is Dig...
1,Thu Nov 09 17:43:40 EST 2017,: Lots of love from unknown miners. Miners are...
2,Thu Nov 09 17:43:39 EST 2017,: Warning: Built-in Keylogger Found in MantisT...
3,Thu Nov 09 17:43:39 EST 2017,: Join our telegram. All information about and...
4,Thu Nov 09 17:43:39 EST 2017,": $DIGAF: FLOAT=16M, THE ""ONLY"" EXCHANGE ON TH..."
...,...,...
2564345,Mon Nov 20 13:40:17 EST 2017,"Bitcoin continues to crush it, breaks through ..."
2564346,Mon Nov 20 13:40:16 EST 2017,: Bitcoin's market cap just passed McDonald's....
2564347,Mon Nov 20 13:40:16 EST 2017,as bitcoin looks to test the waters with
2564348,Mon Nov 20 13:40:16 EST 2017,CME's Bitcoin Futures Likely to Start Trading ...


In [56]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [57]:
analyzer = SentimentIntensityAnalyzer()
tweetsDF['compound'] = [analyzer.polarity_scores(x)['compound'] for x in tweetsDF['Tweet']]
tweetsDF['neg'] = [analyzer.polarity_scores(x)['neg'] for x in tweetsDF['Tweet']]
tweetsDF['neu'] = [analyzer.polarity_scores(x)['neu'] for x in tweetsDF['Tweet']]
tweetsDF['pos'] = [analyzer.polarity_scores(x)['pos'] for x in tweetsDF['Tweet']]

In [58]:
tweetsDF

,DateTime,Tweet,compound,neg,neu,pos
0,Thu Nov 09 17:43:41 EST 2017,: The Failure of SegWit2x Shows Bitcoin is Dig...,-0.6914,0.322,0.678,0.000
1,Thu Nov 09 17:43:40 EST 2017,: Lots of love from unknown miners. Miners are...,0.8750,0.000,0.577,0.423
2,Thu Nov 09 17:43:39 EST 2017,: Warning: Built-in Keylogger Found in MantisT...,-0.3400,0.146,0.854,0.000
3,Thu Nov 09 17:43:39 EST 2017,: Join our telegram. All information about and...,0.2960,0.000,0.784,0.216
4,Thu Nov 09 17:43:39 EST 2017,": $DIGAF: FLOAT=16M, THE ""ONLY"" EXCHANGE ON TH...",0.0000,0.000,1.000,0.000
...,...,...,...,...,...,...
2564345,Mon Nov 20 13:40:17 EST 2017,"Bitcoin continues to crush it, breaks through ...",-0.1531,0.186,0.814,0.000
2564346,Mon Nov 20 13:40:16 EST 2017,: Bitcoin's market cap just passed McDonald's....,0.0000,0.000,1.000,0.000
2564347,Mon Nov 20 13:40:16 EST 2017,as bitcoin looks to test the waters with,0.0000,0.000,1.000,0.000
2564348,Mon Nov 20 13:40:16 EST 2017,CME's Bitcoin Futures Likely to Start Trading ...,0.0000,0.000,1.000,0.000


In [59]:
tweetsDF['DateTime'] = tweetsDF['DateTime'].str[:-9]

In [60]:
tweetsDF['DateTime'] = tweetsDF['DateTime'].str[4:]

In [61]:
tweetsDF['DateTime'] = tweetsDF['DateTime'].str.replace('Nov ','2017-11-')
tweetsDF['DateTime'] = tweetsDF['DateTime'].str.replace('Oct ','2017-10-')

In [62]:
tweetsDF

,DateTime,Tweet,compound,neg,neu,pos
0,2017-11-09 17:43:41,: The Failure of SegWit2x Shows Bitcoin is Dig...,-0.6914,0.322,0.678,0.000
1,2017-11-09 17:43:40,: Lots of love from unknown miners. Miners are...,0.8750,0.000,0.577,0.423
2,2017-11-09 17:43:39,: Warning: Built-in Keylogger Found in MantisT...,-0.3400,0.146,0.854,0.000
3,2017-11-09 17:43:39,: Join our telegram. All information about and...,0.2960,0.000,0.784,0.216
4,2017-11-09 17:43:39,": $DIGAF: FLOAT=16M, THE ""ONLY"" EXCHANGE ON TH...",0.0000,0.000,1.000,0.000
...,...,...,...,...,...,...
2564345,2017-11-20 13:40:17,"Bitcoin continues to crush it, breaks through ...",-0.1531,0.186,0.814,0.000
2564346,2017-11-20 13:40:16,: Bitcoin's market cap just passed McDonald's....,0.0000,0.000,1.000,0.000
2564347,2017-11-20 13:40:16,as bitcoin looks to test the waters with,0.0000,0.000,1.000,0.000
2564348,2017-11-20 13:40:16,CME's Bitcoin Futures Likely to Start Trading ...,0.0000,0.000,1.000,0.000


In [63]:
tweetsDF['DateTime'] = pd.to_datetime(tweetsDF['DateTime'], format='%y-%m-%d %H:%M:%S', infer_datetime_format=True)

In [21]:
btcDF = pd.read_csv('BitCoinPrice.csv', encoding='ISO-8859-1', error_bad_lines = False, engine = 'python')

In [22]:
btcDF['Date'] = pd.to_datetime(btcDF['Date'], format='%y-%m-%d %H:%M:%S', infer_datetime_format=True)

In [23]:
btcDF

,Date,Close Price
0,2017-10-30 00:00:00,6123.21
1,2017-10-30 01:00:00,6131.35
2,2017-10-30 02:00:00,6114.17
3,2017-10-30 03:00:00,6153.11
4,2017-10-30 04:00:00,6151.09
...,...,...
667,2017-11-26 19:00:00,9233.84
668,2017-11-26 20:00:00,9304.96
669,2017-11-26 21:00:00,9351.25
670,2017-11-26 22:00:00,9337.11


In [24]:
grouped2H = tweetsDF.groupby(pd.Grouper(freq='2H', key='DateTime')).mean()

In [25]:
grouped2H

,compound,neg,neu,pos
DateTime,,,,
2017-10-31 00:00:00,0.125369,0.028529,0.875785,0.094359
2017-10-31 02:00:00,0.118621,0.032672,0.870046,0.096681
2017-10-31 04:00:00,0.140240,0.034335,0.857123,0.108276
2017-10-31 06:00:00,0.124437,0.029499,0.871936,0.098077
2017-10-31 08:00:00,0.125001,0.028730,0.877517,0.092991
...,...,...,...,...
2017-11-17 12:00:00,0.088819,0.051375,0.854360,0.091349
2017-11-17 14:00:00,0.101108,0.048070,0.853205,0.095925
2017-11-17 16:00:00,0.136924,0.038493,0.860657,0.100323


In [26]:
btcDF_2H = btcDF.groupby(pd.Grouper(key='Date', freq='2H'))['Close Price'].mean().reset_index(name='#Close Price')
btcDF_2H

,Date,#Close Price
0,2017-10-30 00:00:00,6127.280
1,2017-10-30 02:00:00,6133.640
2,2017-10-30 04:00:00,6134.595
3,2017-10-30 06:00:00,6135.710
4,2017-10-30 08:00:00,6153.330
...,...,...
331,2017-11-26 14:00:00,9056.175
332,2017-11-26 16:00:00,9278.980
333,2017-11-26 18:00:00,9357.560
334,2017-11-26 20:00:00,9328.105


In [27]:
tweet_bitcoin2H = pd.merge(grouped2H, btcDF_2H,left_on='DateTime', right_on='Date')
tweet_bitcoin2H

,compound,neg,neu,pos,Date,#Close Price
0,0.125369,0.028529,0.875785,0.094359,2017-10-31 00:00:00,6140.965
1,0.118621,0.032672,0.870046,0.096681,2017-10-31 02:00:00,6129.460
2,0.140240,0.034335,0.857123,0.108276,2017-10-31 04:00:00,6151.340
3,0.124437,0.029499,0.871936,0.098077,2017-10-31 06:00:00,6100.130
4,0.125001,0.028730,0.877517,0.092991,2017-10-31 08:00:00,6145.065
...,...,...,...,...,...,...
210,0.088819,0.051375,0.854360,0.091349,2017-11-17 12:00:00,7791.230
211,0.101108,0.048070,0.853205,0.095925,2017-11-17 14:00:00,7933.965
212,0.136924,0.038493,0.860657,0.100323,2017-11-17 16:00:00,7860.955
213,0.112747,0.041463,0.863354,0.094179,2017-11-17 18:00:00,7826.990


In [29]:
cols = list(tweet_bitcoin2H)
cols

['compound', 'neg', 'neu', 'pos', 'Date', '#Close Price']

In [30]:
cols.insert(0, cols.pop(cols.index('Date')))
cols

['Date', 'compound', 'neg', 'neu', 'pos', '#Close Price']

In [31]:
tweet_bitcoin2H = tweet_bitcoin2H.loc[:, cols]
tweet_bitcoin2H

,Date,compound,neg,neu,pos,#Close Price
0,2017-10-31 00:00:00,0.125369,0.028529,0.875785,0.094359,6140.965
1,2017-10-31 02:00:00,0.118621,0.032672,0.870046,0.096681,6129.460
2,2017-10-31 04:00:00,0.140240,0.034335,0.857123,0.108276,6151.340
3,2017-10-31 06:00:00,0.124437,0.029499,0.871936,0.098077,6100.130
4,2017-10-31 08:00:00,0.125001,0.028730,0.877517,0.092991,6145.065
...,...,...,...,...,...,...
210,2017-11-17 12:00:00,0.088819,0.051375,0.854360,0.091349,7791.230
211,2017-11-17 14:00:00,0.101108,0.048070,0.853205,0.095925,7933.965
212,2017-11-17 16:00:00,0.136924,0.038493,0.860657,0.100323,7860.955
213,2017-11-17 18:00:00,0.112747,0.041463,0.863354,0.094179,7826.990


In [32]:
tweet_bitcoin2H.to_csv(r'E:\DLKD\PROJECT\vader2H.csv')

In [64]:
grouped1H = tweetsDF.groupby(pd.Grouper(freq='1H', key='DateTime')).mean()

In [65]:
grouped1H

,compound,neg,neu,pos
DateTime,,,,
2017-10-31 00:00:00,0.111746,0.030924,0.878599,0.088505
2017-10-31 01:00:00,0.139266,0.026086,0.872914,0.100331
2017-10-31 02:00:00,0.122860,0.029641,0.872456,0.097270
2017-10-31 03:00:00,0.114767,0.035427,0.867855,0.096145
2017-10-31 04:00:00,0.112043,0.038130,0.861398,0.099918
...,...,...,...,...
2017-11-27 13:00:00,0.099922,0.042160,0.864631,0.091561
2017-11-27 14:00:00,0.105359,0.037827,0.869259,0.092086
2017-11-27 15:00:00,0.113607,0.041003,0.859230,0.098482


In [35]:
btcDF_1H = btcDF.groupby(pd.Grouper(key='Date', freq='1H'))['Close Price'].mean().reset_index(name='Price')
btcDF_1H

,Date,Price
0,2017-10-30 00:00:00,6123.21
1,2017-10-30 01:00:00,6131.35
2,2017-10-30 02:00:00,6114.17
3,2017-10-30 03:00:00,6153.11
4,2017-10-30 04:00:00,6151.09
...,...,...
667,2017-11-26 19:00:00,9233.84
668,2017-11-26 20:00:00,9304.96
669,2017-11-26 21:00:00,9351.25
670,2017-11-26 22:00:00,9337.11


In [66]:
tweet_bitcoin1H = pd.merge(grouped1H, btcDF_1H,left_on='DateTime', right_on='Date')

In [67]:
tweet_bitcoin1H

,compound,neg,neu,pos,Date,Price
0,0.111746,0.030924,0.878599,0.088505,2017-10-31 00:00:00,6142.46
1,0.139266,0.026086,0.872914,0.100331,2017-10-31 01:00:00,6139.47
2,0.122860,0.029641,0.872456,0.097270,2017-10-31 02:00:00,6128.20
3,0.114767,0.035427,0.867855,0.096145,2017-10-31 03:00:00,6130.72
4,0.112043,0.038130,0.861398,0.099918,2017-10-31 04:00:00,6143.92
...,...,...,...,...,...,...
643,0.137856,0.034510,0.868098,0.096605,2017-11-26 19:00:00,9233.84
644,0.126233,0.032412,0.875534,0.090744,2017-11-26 20:00:00,9304.96
645,0.144791,0.027903,0.875409,0.095615,2017-11-26 21:00:00,9351.25
646,0.132963,0.031965,0.876167,0.091420,2017-11-26 22:00:00,9337.11


In [68]:
col = list(tweet_bitcoin1H)
col

['compound', 'neg', 'neu', 'pos', 'Date', 'Price']

In [69]:
col.insert(0, col.pop(col.index('Date')))
col

['Date', 'compound', 'neg', 'neu', 'pos', 'Price']

In [70]:
tweet_bitcoin1H = tweet_bitcoin1H.loc[:, col]
tweet_bitcoin1H

,Date,compound,neg,neu,pos,Price
0,2017-10-31 00:00:00,0.111746,0.030924,0.878599,0.088505,6142.46
1,2017-10-31 01:00:00,0.139266,0.026086,0.872914,0.100331,6139.47
2,2017-10-31 02:00:00,0.122860,0.029641,0.872456,0.097270,6128.20
3,2017-10-31 03:00:00,0.114767,0.035427,0.867855,0.096145,6130.72
4,2017-10-31 04:00:00,0.112043,0.038130,0.861398,0.099918,6143.92
...,...,...,...,...,...,...
643,2017-11-26 19:00:00,0.137856,0.034510,0.868098,0.096605,9233.84
644,2017-11-26 20:00:00,0.126233,0.032412,0.875534,0.090744,9304.96
645,2017-11-26 21:00:00,0.144791,0.027903,0.875409,0.095615,9351.25
646,2017-11-26 22:00:00,0.132963,0.031965,0.876167,0.091420,9337.11


In [71]:
tweet_bitcoin1H.to_csv(r'E:\DLKD\PROJECT\vader1H_final.csv')